In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import datetime
load_dotenv()

True

In [28]:
llm_model = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash',
                                    temperature = 0,
                                    max_tokens=None,
                                    timeout=None,
                                    max_retries=2)

In [40]:
##Actor agent Prompt
##Actor is the main component -->> Subcomponents are 1. Response, Revisor
##chatprompt template is a list of tuples

##first instruction -->> example : write a blog post in 250 words (user query)
## 2nd instruction -->> reflect & critique the answer 
## 3rd instruction -->> provide search queries
actor_prompt_template = ChatPromptTemplate.from_messages([
    ("system",
     """ You are an expert AI researcher.
     Current time : {time}
     1. {first_instruction}
     2. Reflect and critique your answer. Be severe to maximize improvement.
     3. After the reflection, **list 1-3 search queries separately** for researching improvements. Do this separately, outside reflection step.
     """),
    MessagesPlaceholder(variable_name="messages"),
    #("system","Answer the user's question above using the required format")
]
).partial(time=lambda: datetime.datetime.now().isoformat()) ##dynamic time stamp, lambda doest need arguments

In [41]:
## LLM parser system : Response from unstructured LLM <<-->> structured Output ##
###schema###
from pydantic import BaseModel, Field
from typing import List

class Reflection(BaseModel):
    missing: str = Field(description='Critique of what is missing.')
    superflous: str = Field(description='Critique of what is superflous')

class AnswerQuestion(BaseModel):
    answer: str = Field(description="~250 word detailed answer to the question.")
    search_queries: List[str] = Field(description="1-3 search queries for researching the improvements to address,\
                                                    the critique of your current answer.")
    reflection: Reflection = Field(description = 'Your reflection on the initial answer')
    
    #### this field is itself an instance of Reflection base model ###
    #### internally convert to Relfection base model####
    '''
    Ex: data = AnswerQuestion(reflection={'missing':,search_queries:''}),
        which pydantic converts into
        data = AnswerQuestion(reflection = Reflection(missing=,search_queries=))
    '''
    ###Fields -->> Extra Metadata, Constraints for the field, gt,ge can be used, ... means required field

In [42]:
### responder chain
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
pydantic_parser = PydanticToolsParser(tools = [AnswerQuestion])
###Pydantic parser grounds the LLM output into the schema, does data validation(whether llm output follows the required format or not####
first_responder_prompt_template = actor_prompt_template.partial(first_instruction="Provide a detailed ~250 word answer")
first_responder_chain = first_responder_prompt_template | llm_model.bind_tools(tools=[AnswerQuestion],
                                                                        tool_choice='AnswerQuestion') | pydantic_parser

In [43]:
response = first_responder_chain.invoke({"messages":[HumanMessage(content="Write a blog post in not more than 250 words on how,\
                                                                small businesses can leverage AI to grow")]
                                        })
print(response)

[AnswerQuestion(answer='## AI: The Small Business Game Changer\n\nSmall businesses often struggle to compete with larger corporations, but Artificial Intelligence (AI) is leveling the playing field. AI offers affordable solutions to boost efficiency and drive growth.\n\n**Customer Service:** AI-powered chatbots provide instant customer support, freeing up staff for other tasks. They can answer frequently asked questions, resolve simple issues, and even generate leads.\n\n**Marketing:** AI algorithms analyze customer data to personalize marketing campaigns. This targeted approach increases engagement and conversion rates, maximizing your marketing ROI.\n\n**Operations:** AI can automate repetitive tasks like data entry and invoice processing, saving time and reducing errors. AI-powered tools can also optimize inventory management and predict demand.\n\n**Sales:** AI can analyze sales data to identify trends and opportunities. This information can be used to improve sales strategies and 